In [1]:
# Загружаем необходимые библиотеки и классы (treeinterpreter можно 
# установить с помощью команды pip install treeinterpreter)
from treeinterpreter import treeinterpreter as ti
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

In [2]:
# Загружаем данные для задачи регрессии
from sklearn.datasets import load_boston
boston = load_boston()
# Подгоняем ансамбль деревьев регрессии
rf = RandomForestRegressor(random_state=42)
rf.fit(boston.data, boston.target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [3]:
# Отдельно запишем наблюдение, для которого будет смотреть прогноз
observation=boston.data[300:301]

In [4]:
# Печатаем прогноз для наблюдения
print("Прогноз для наблюдения:", rf.predict(observation))

Прогноз для наблюдения: [ 26.36]


In [5]:
# Выполним декомпозицию прогноза для наблюдения
prediction, bias, contributions = ti.predict(rf, observation)
# Печатаем результаты декомпозиции прогноза
print("Прогноз", prediction)
print("Смещение (среднее зависимой переменной в обучающем наборе)", bias)
print("Вклады признаков:")
for c, feature in zip(contributions[0], boston.feature_names):
    print(feature, c)

Прогноз [ 26.36]
Смещение (среднее зависимой переменной в обучающем наборе) [ 22.63029644]
Вклады признаков:
CRIM -0.336032227032
ZN -0.0375897435897
INDUS 0.215891154189
CHAS 0.0
NOX 0.251317322288
RM 2.91335204277
AGE -0.641340080972
DIS -1.57627007116
RAD -0.042
TAX -0.301164860662
PTRATIO 0.00182041924623
B -0.0351495726496
LSTAT 3.31686917489


In [6]:
# Печатаем результаты декомпозиции прогноза, отсортировав 
# вклады признаков по убыванию абсолютного значения и 
# выполнив округление до второго знака
print("Смещение (среднее зависимой переменной в обучающем наборе)", bias)
print("Вклады признаков:")
for c, feature in sorted(zip(contributions[0], 
                             boston.feature_names), 
                             key=lambda x: -abs(x[0])):
    print(feature, round(c, 2))    

Смещение (среднее зависимой переменной в обучающем наборе) [ 22.63029644]
Вклады признаков:
LSTAT 3.32
RM 2.91
DIS -1.58
AGE -0.64
CRIM -0.34
TAX -0.3
NOX 0.25
INDUS 0.22
RAD -0.04
ZN -0.04
B -0.04
PTRATIO 0.0
CHAS 0.0


In [7]:
# Сложим смещение и вклады признаков, чтобы проверить,
# совпадает ли полученный результат с прогнозом
print(prediction)
print(bias + np.sum(contributions, axis=1))

[ 26.36]
[ 26.36]


In [8]:
# Загружаем данные для задачи классификации в пандасовский DataFrame
data = pd.read_csv("C:/Trees/BankloanPy.csv", encoding='cp1251', sep=';')
# Формируем массив признаков и массив меток
y = data.loc[:, 'default']
X = data.loc[:, 'age':'othdebt']
# Записываем названия признаков, чтобы потом сопоставить их значениям вкладов
features_names=X.columns
# Разбиваем на обучение и тест
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Подгоняем ансамбль деревьев классификации
forest=RandomForestClassifier(n_estimators=200, random_state=42)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [9]:
# Печатаем результаты декомпозиции прогноза для конкретного наблюдения
prediction_test, bias_test, contributions_test = ti.predict(forest, X_test[100:101])
print("Прогноз", prediction_test)
print("Смещения (вероятности классов зависимой переменной в обучающем наборе)", bias_test)
print("Вклады признаков:")
for c, feature in zip(contributions_test[0], features_names):
    print(feature, c)

Прогноз [[ 0.95  0.05]]
Смещения (вероятности классов зависимой переменной в обучающем наборе) [[ 0.64397143  0.35602857]]
Вклады признаков:
age [ 0.04751199 -0.04751199]
employ [ 0.07158361 -0.07158361]
address [ 0.0427569 -0.0427569]
income [ 0.04874334 -0.04874334]
debtinc [ 0.07574518 -0.07574518]
creddebt [ 0.00998228 -0.00998228]
othdebt [ 0.00970526 -0.00970526]
